In [3]:
import numpy as np
from scipy.linalg import solve_banded

In [23]:
def a(i, j, epsilon, x0, x1, x2):
    """
    Paramaters:
        i (int): index of phi function
        j (int): index of phi function
        epsilon (float): constant
        interval (tuple): (x0, x1, x3) partition x values
    Returns:
        result (float): evaluation of function 
    """
    interval = (x0, x1, x2)
    h_i = interval[1] - interval[0]
    h_i_1 = interval[2] - interval[1]
    if j == i+1:
        return epsilon/h_i_1 + 1/2
    elif i == j:
        return -epsilon/h_i - epsilon/h_i_1
    elif j == i-1:
        return epsilon/h_i - 1/2
    else:
        return 0
    
def l(j, x0, x1, x2):
    """
    Parameters:
        j (int): index of phi function
        interval (tuple): (x0, x1, x3) partition x values 
    Returns:
        result (float): value of l function
    """
    interval = (x0, x1, x2)
    h_j = interval[1] - interval[0]
    h_j_1 = interval[2] - interval[1]
    return -1/2*(h_j + h_j_1)

l = np.vectorize(l)
a = np.vectorize(a)

In [91]:
def finite_element(epsilon, alpha, beta, xmin, xmax, N):
    """
    Parameters:
        epsilon (float): constant
        alpha (float): y(xmin) = alpha
        beta (float): y(xmax) = beta
        xmax (float): upper bound for x
        xmin (float): lower bound for x
        N (int): number of intervals (N+1 total points)
    """
    # Number of non-zero lower and upper diagonals for solved_banded function
    u_ = 1
    l_ = 1
    
    xs = np.linspace(xmin, xmax, N+1)
    dx = (xmax - xmin) / N
    intervals = [(x - dx, x, x + dx) for x in xs]

#     tuple1 = [(i+2, interval) for i,interval in enumerate(intervals[2:N+1])]
#     tuple2 = [(i+1, interval) for i,interval in enumerate(intervals[1:N+1])]
#     tuple3 = [(i, interval) for i,interval in enumerate(intervals[0:N-1])]
    
    # Construct ab matrix for solve_banded function see documentation for details
    row1 = np.concatenate((np.array([0]), np.array([0]), np.array([a(i+2, i+1, epsilon, *interval) for i, interval in enumerate(intervals[2:N+1])])))
    row2 = np.concatenate((np.array([1]), np.array([a(i+1, i+1, epsilon, *interval) for i, interval in enumerate(intervals[1:N])]), np.array([1])))
    row3 = np.concatenate((np.array([a(i, i+1, epsilon, *interval) for i, interval in enumerate(intervals[:N-1])]), np.array([0]), (np.array([0]))))
    
#     print(row1.shape, row2.shape, row3.shape)
    
    ab = np.vstack((row1, row2, row3))
    
    # Set up b array and empty solution vector
    b = np.array([l(j, interval[0], interval[1], interval[2]) for j, interval in enumerate(intervals[1:N])])
    b = np.concatenate((np.array([alpha]), b, np.array([beta])))
    sol = np.zeros(N+1)
    
#     print(ab.shape, b.shape)
#     print(u_, l_)
#     print(ab, b)
    
    # Solve banded function will now solve our ax = b problem
    sol = solve_banded((l_,u_), ab, b)
    
    return sol
    

In [99]:
alpha, beta, epsilon, N = 2, 4, 0.02, 100

y = lambda x : alpha + x + (beta - alpha - 1) * (np.exp(x/epsilon)) / (np.exp(1/epsilon) - 1)

print(np.allclose(finite_element(epsilon, alpha, beta, xmin, xmax, N), y(np.linspace(0, 1, 101)), atol=0.01))

True
